* Dataset
* Build model
* Define loss_fn & optimizer
* Define trainer (model produces predections -> compute the loss (label - pred ) -> backprop)
* Define test fun (on validation) -> trainin/validation/test
* Run the trainer and test

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import pytorch_lightning as pl


In [ ]:
# get data set

train_ds= MNIST(root='data', train=True, download=True, transform=ToTensor())
valid_ds= MNIST(root='data', train=False, download=True, transform=ToTensor())

bs =64
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=True)

In [ ]:
#Build MOdel

import pytorch_lightning as pl
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchmetrics

class MNISTModel(pl.LightningModule):
    def __init__(self, lr=0.5):  # Fixed typo in the method name
        super().__init__()
        self.lr = lr
        self.lin = nn.Linear(784, 10)
        self.train_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=10)
        self.valid_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=10)


    def forward(self, xb):  # Fixed typo in the method name
        xb = xb.flatten(1, -1)
        return self.lin(xb)

    def training_step(self, batch, batch_idx):
        loss = self.shared_step(batch, train=True)
        return loss

    def validation_step(self, batch, batch_idx):
        self.shared_step(batch, train=False)

    def shared_step(self, batch, train):
        xb, yb = batch
        pred = self(xb)
        loss = F.cross_entropy(pred, yb)

        if (train):
            self.train_accuracy(pred.softmax(dim=-1), yb)
            self.log('train_accuracy', self.train_accuracy, on_step=True, on_epoch=False, prog_bar=True)
        else:
            self.valid_accuracy(pred.softmax(dim=-1), yb)
            self.log('valid_accuracy', self.train_accuracy, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=self.lr)  # Fixed typo in the parameter name


In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

# logger
tb_logger= TensorBoardLogger('tb_logs')

In [ ]:
# initilaize model

mnist_model= MNISTModel()

# init trainer

trainer= pl.Trainer(max_epochs=2, logger=tb_logger)

# Train model
trainer.fit(mnist_model, train_dl, valid_dl)
